In [1]:
import configparser
import os
import sys
from time import sleep
from uuid import uuid4

import numpy as np

import GCode
import GRBL

# Laser Shading By adjusting focus on the Z-axis

# Code:

In [2]:
# Poplar 1x4". Cut
BlockHeight = 89.0  # [mm]
BlockLength = 2 * BlockHeight  # [mm]
BlockThickness = 19.0  # [mm]
print("{} x {} x {}".format(BlockHeight, BlockLength, BlockThickness))


89.0 x 178.0 x 19.0


In [3]:
config = configparser.ConfigParser()
config["GRBL"] = dict()
config["GRBL"]["port"] = "/dev/cnc_3018"
cnc = GRBL.GRBL(**config["GRBL"])

cnc.reset(home=True)

(0, ['ok', '', "Grbl 1.1f ['$' for help]", 'ok'])

In [4]:
if "Alarm" in cnc.status:
    cnc.home()

# Generate Program

Generate the G-code program that will test how the laser focus affects shading.

- Laser 'focused' at -14mm.
- Testing 17 positions between [-28, 0]
- Testing 3 different feed rates [100, 200, 300]
- Testing 5 different power settings between [100, 255]

In [5]:
# Define a gap between horizontal lines
vgap = 5  # [mm]
# Generate a list of horizontal line vertical coordinates
vs = np.arange(vgap, BlockHeight, vgap)

# Define a horizontal distance to test lazerin.
hgap = 10  # [mm]
hs = np.arange(0, BlockLength, hgap)

# Total travel range, starts at ztravel, ends at 0. "negative space"(?)
ztravel = -28
# Generate a number of z-axis test positions.
# Each z-height will have 10 mm marked at the given feed rate and power.

# Generate the full list of z-stops from the initial test.
zs0 = np.linspace(start=ztravel, stop=0, num=len(hs) - 1, endpoint=True)

# Laser powers to test.
powers = [255, 200, 150]

feeds = [50, 100, 150, 200, 300]

In [6]:
def init2(**kwargs):
    program = GCode.GCode(**kwargs)
    program.G0(F=500)
    program.G1(F=500)
    program.G21()  # Metric Units
    program.G90()  # Absolute positioning.
    return program

In [7]:
def aim(**kwargs):
    program = GCode.GCode(**kwargs)
    program.G0(F=500)
    program.G1(F=300)
    program.M3(S=1)
    program.G21()  # Metric Units
    program.G90()  # Absolute positioning.
    program.G1(X=10, Y=10, Z=-14.0)
    return program

In [8]:
cnc.init()

True

In [9]:
aim(machine=cnc).run()

In [11]:
programs=list()
for idx1, power in enumerate([255]): # [uint8 duty cycle]
    for idx2, feed in enumerate([50]):  # [mm/min]
        v_idx = idx1 * len(powers) + idx2
        Y = vs[v_idx]
        for idx3, z in enumerate(zs0):
            x0 = hs[idx3]
            xf = hs[idx3 + 1]
            
            program = init2(machine=cnc)
            program.G0(Z=z, Y=Y, X=x0)
            program.M4(S=power)
            program.G1(X=xf, Y=Y, Z=z, F=feed)
            program.M5()  # Laser. Off.
            programs.append(program)

In [12]:
cnc.init()

True

In [13]:
for idx, program in enumerate(programs):
    print("{}: ".format(idx), end="")
    print(cnc.run(program))

0: 5.14356803894043
1: 13.62215805053711
2: 13.622452974319458
3: 13.621125936508179
4: 13.621455669403076
5: 13.621223211288452
6: 13.622615337371826
7: 13.621844291687012
8: 13.62470531463623
9: 13.621888399124146
10: 13.621965169906616
11: 13.621423482894897
12: 13.62158203125
13: 13.621318578720093
14: 13.621687889099121
15: 13.624190330505371
16: 13.620553970336914


In [15]:
len(zs0)

17